In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
import numba
import pyvista as pv
import pywavefront
import os
from pathlib import Path

In [ ]:
from primitives.triangle import Triangle
from scenes.scene import Scene
from scenes.render import render_scene
from utils.misc import get_floor

In [ ]:
#load object
path = Path("objects/cube.obj").resolve()
mesh = pv.read(path)
mesh.plot()

In [ ]:
mesh = mesh.triangulate()
points = np.ascontiguousarray(mesh.points)
points

In [ ]:
# new_position = -1 * (points.max(axis=0))
# new_position

In [ ]:
# mesh.points -= mesh.center
# mesh.points += new_position
# mesh.points

In [ ]:
points = np.ascontiguousarray(mesh.points)
faces = mesh.faces.reshape((-1,4))[:, 1:4]
vertices = np.ascontiguousarray(points[faces], dtype=np.float64)

In [ ]:
triangles = numba.typed.List()
# triangles = get_floor(x_depth, y_depth, z_depth)
triangles

In [ ]:

for v in vertices:
    a,b,c = v[0], v[1], v[2]

    triangle = Triangle(vertex_1=np.ascontiguousarray(a, dtype=np.float64),
                        vertex_2=np.ascontiguousarray(b, dtype=np.float64),
                        vertex_3=np.ascontiguousarray(c, dtype=np.float64)
                        )
    triangles.append(triangle)

len(triangles)

In [ ]:
# Setup scene

# point camera
look_at = np.array([0, 0, 1], dtype=np.float64)
camera = np.array([1, 1, -5], dtype=np.float64)

# screen
width=300
height=300


# Initialise the scene with lights and camera
scene = Scene(look_at=look_at, camera=camera, width=width, height=height)

In [ ]:
from accelerators.bvh import BoundedBox, build_bvh, LinearBVHNode, flatten_bvh

# Enclose all the primitives in their individual axis-aligned bounding boxes (AABB)
bounded_boxes = []
for i in range(len(triangles)):
    bounded_boxes.append(BoundedBox(triangles[i], i))

len(triangles), len(bounded_boxes)

In [ ]:
# Initialise a bounding volume hierarchy tree
start = 0
end = len(bounded_boxes)
ordered_prims = []
total_nodes = 0
root, bounded_boxes, ordered_prims, total_nodes = build_bvh(triangles, bounded_boxes, start, end, ordered_prims, total_nodes)
# swap the primitives array with the ordered list
primitives = ordered_prims


# create a linear representation of the bvh tree
linear_nodes = numba.typed.List()
for _ in range(total_nodes):
    linear_nodes.append(LinearBVHNode())


# linear bvh
linear_bvh, off_set = flatten_bvh(linear_nodes, root, 0)
# len(linear_bvh)

# check if indexes of all primitives present in the linear bvh tree
total = 0
for _bvh in linear_bvh:
    total+= _bvh.n_primitives
print(total==len(triangles)) # should be equal to the number of primitives


In [ ]:
start = time.time()
# primitives = spheres, triangles
image = render_scene(scene, triangles, linear_bvh)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

In [ ]:
plt.imshow(image)